# Load Data

In [21]:
import networkx as nx

mobility_network = nx.read_gml("data/Mobility.gml")
communication_network = nx.read_gml("data/Communication.gml")

# Add Geocodes

In [83]:
import urllib3
import json

# setup HTTP pool with custom user-agent (required by Nominatim)
http = urllib3.PoolManager(num_pools=1, headers={'User-Agent': 'my-test-app'})

def fetch_geocode(lat, lon):
    """
    fetch reverse geocoding data from OSM Nominatim.
    """
    url = f'https://nominatim.openstreetmap.org/reverse.php?lat={lat}&lon={lon}&zoom=18&format=jsonv2'
    try:
        resp = http.request('GET', url, timeout=urllib3.Timeout(connect=5.0, read=10.0))
        if resp.status == 200:
            return json.loads(resp.data.decode())
        else:
            print(f"⚠️ Failed for ({lat}, {lon}) - status {resp.status}")
            return None
    except Exception as e:
        print(f"⚠️ Error for ({lat}, {lon}): {e}")
        return None

def enrich_graph(graph):
    """
    iterate over graph nodes,enrich with reverse geocode info.
    """
    for node, data in graph.nodes(data=True):
        # skip if already enriched
        if "country" in data:
            continue
        result = fetch_geocode(lat=data.get("lat"), lon=data.get("lon"))
        if result:
            address = result.get("address", {})
            country = address.get("country", "")
            state = address.get("state", "")
            county = address.get("county", "")
            city = address.get("city", "")

            graph.nodes[node]["country"] = country
            graph.nodes[node]["state"] = state
            graph.nodes[node]["county"] = county
            graph.nodes[node]["city"] = city

enrich_graph(communication_network)
enrich_graph(mobility_network)

# Save Augmented Graph

In [84]:
nx.write_gml(communication_network, "data/Communication_Augmented.gml")
nx.write_gml(mobility_network, "data/Mobility_Augmented.gml")